In [1]:
from keras.initializers import glorot_uniform
from keras.layers import Input, ZeroPadding2D, Add, GlobalAveragePooling2D, Concatenate, Conv2DTranspose, Multiply
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Activation, Flatten, Dropout, Dense, AveragePooling2D, Lambda, Softmax
from keras.models import Sequential, Model
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.utils import to_categorical
from keras.utils.data_utils import get_file
from sklearn.metrics import confusion_matrix, classification_report
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings 
import tensorflow as tf
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')



In [2]:
data = pd.read_csv(r'../input/dl-fer2013-final/fer2013_with_mask_final.csv')
training_set,validation_set = train_test_split(data,test_size=0.2,shuffle=True)
training_labels = to_categorical(training_set['emotion'])

validation_labels = to_categorical(validation_set['emotion'])

training_pixels = training_set['pixels'].str.split().tolist()
training_pixels = np.array(training_pixels)

training_pixels = training_pixels.reshape(-1, 48, 48, 3)
training_pixels = training_pixels.astype(np.float32) / 255
#training_pixels = np.repeat(training_pixels, 3, -1)        # convert to 3channels



validation_pixels = validation_set['pixels'].str.split().tolist()
validation_pixels = np.array(validation_pixels)
validation_pixels = validation_pixels.reshape(-1, 48, 48, 3)
validation_pixels = validation_pixels.astype(np.float32) / 255
#validation_pixels = np.repeat(validation_pixels, 3, -1)        # convert to 3channels

aug = ImageDataGenerator(
    rotation_range=25, shear_range=0.2, 
    zoom_range=0.2,horizontal_flip=True, 
    fill_mode="nearest")

In [3]:
print(training_pixels.shape)

In [4]:
def masking(X, f, stage, block, depth):
  filters = [f, f * 2, f * 4, f * 8, f * 16]
  
  if depth == 1:
    X = conv_block(X, filters[1], stage, block + 1, strides=(1, 1))
    X = conv_block(X, filters[0], stage, block + 2, strides=(1, 1))
  elif depth == 2:
    X1 = conv_block(X, filters[1], stage, block + 1, strides=(1, 1))
    P1 = MaxPooling2D((2, 2))(X1)
    X2 = conv_block(P1, filters[2], stage, block + 2, strides=(1, 1))
  
    X3 = Conv2DTranspose(filters[1], kernel_size=(2, 2), strides=(2, 2))(X2)
    X3 = BatchNormalization(axis=3)(X3)
    X3 = Activation('relu')(X3)
    X3 = Concatenate(axis=-1)([X3, X1])
    X3 = conv_block(X3, filters[1], stage, block + 3, strides=(1, 1))

    X = conv_block(X3, filters[0], stage, block + 4, strides=(1, 1))
  elif depth == 3:
    X1 = conv_block(X, filters[1], stage, block + 1, strides=(1, 1))
    P1 = MaxPooling2D((2, 2))(X1)
    X2 = conv_block(P1, filters[2], stage, block + 2, strides=(1, 1))
    P2 = MaxPooling2D((2, 2))(X2)
    X3 = conv_block(P2, filters[3], stage, block + 3, strides=(1, 1))

    X4 = Conv2DTranspose(filters[2], kernel_size=(2, 2), strides=(2, 2))(X3)
    X4 = BatchNormalization(axis=3)(X4)
    X4 = Activation('relu')(X4)
    X4 = Concatenate(axis=-1)([X4, X2])
    X4 = conv_block(X4, filters[2], stage, block + 4, strides=(1, 1))

    X5 = Conv2DTranspose(filters[1], kernel_size=(2, 2), strides=(2, 2))(X4)
    X5 = BatchNormalization(axis=3)(X5)
    X5 = Activation('relu')(X5)
    X5 = Concatenate(axis=-1)([X5, X1])
    X5 = conv_block(X5, filters[1], stage, block + 5, strides=(1, 1))

    X = conv_block(X5, filters[0], stage, block + 6, strides=(1, 1))
  elif depth == 4:
    X1 = conv_block(X, filters[1], stage, block + 1, strides=(1, 1))
    P1 = MaxPooling2D((2, 2))(X1)
    X2 = conv_block(P1, filters[2], stage, block + 2, strides=(1, 1))
    P2 = MaxPooling2D((2, 2))(X2)
    X3 = conv_block(P2, filters[3], stage, block + 3, strides=(1, 1))
    P3 = MaxPooling2D((2, 2))(X3)
    X4 = conv_block(P3, filters[4], stage, block + 4, strides=(1, 1))

    X5 = Conv2DTranspose(filters[3], kernel_size=(2, 2), strides=(2, 2), padding='same')(X4)
    X5 = BatchNormalization(axis=3)(X5)
    X5 = Activation('relu')(X5)
    X5 = Concatenate(axis=-1)([X5, X3])
    X5 = conv_block(X5, filters[3], stage, block + 5, strides=(1, 1))

    X6 = Conv2DTranspose(filters[2], kernel_size=(2, 2), strides=(2, 2), padding='same')(X5)
    X6 = BatchNormalization(axis=3)(X6)
    X6 = Activation('relu')(X6)
    X6 = Concatenate(axis=-1)([X6, X2])
    X6 = conv_block(X6, filters[2], stage, block + 6, strides=(1, 1))

    X7 = Conv2DTranspose(filters[1], kernel_size=(2, 2), strides=(2, 2), padding='same')(X6)
    X7 = BatchNormalization(axis=3)(X7)
    X7 = Activation('relu')(X7)
    X7 = Concatenate(axis=-1)([X7, X1])
    X7 = conv_block(X7, filters[1], stage, block + 7, strides=(1, 1))

    X = conv_block(X7, filters[0], stage, block + 8, strides=(1, 1))
  
  X = Softmax(axis=-1)(X)
  return X
def identity_block(X, f, stage, block):
  name_base = 'stage{}_unit{}_'.format(stage + 1, block + 1)
  conv_name = name_base + 'conv'
  bn_name = name_base + 'bn'
  relu_name = name_base + 'relu'
  sc_name = name_base + 'sc'

  X_shortcut = X

  X = BatchNormalization(axis=3, name=bn_name + '1')(X)
  X = Activation('relu', name=relu_name + '1')(X) 
  X = Conv2D(filters=f, kernel_size=(3, 3), strides=(1, 1), padding='same', use_bias=False, name=conv_name + '1')(X)

  X = BatchNormalization(axis = 3, name=bn_name + '2')(X)
  X = Activation('relu', name=relu_name + '2')(X) 
  X = Conv2D(filters=f, kernel_size=(3, 3), strides=(1, 1), padding='same', use_bias=False, name=conv_name + '2')(X)


  X = Add()([X, X_shortcut])
  return X

def conv_block(X, f, stage, block, strides=(2, 2)):
  name_base = 'stage{}_unit{}_'.format(stage + 1, block + 1)
  conv_name = name_base + 'conv'
  bn_name = name_base + 'bn'
  relu_name = name_base + 'relu'
  sc_name = name_base + 'sc'

  X = BatchNormalization(axis=3, name=bn_name + '1')(X)
  X = Activation('relu', name=relu_name + '1')(X) 

  X_shortcut = Conv2D(filters=f, kernel_size=(1, 1), strides=strides, padding='same', use_bias=False, name=sc_name)(X)

  X = Conv2D(filters=f, kernel_size=(3, 3), strides=strides, padding='same', use_bias=False, name=conv_name + '1')(X)

  X = BatchNormalization(axis = 3, name=bn_name + '2')(X)
  X = Activation('relu', name=relu_name + '2')(X) 
  X = Conv2D(filters=f, kernel_size=(3, 3), strides=(1, 1), padding='same', use_bias=False, name=conv_name + '2')(X)


  X = Add()([X, X_shortcut])
  return X
def ResMaskingNet(input_tensor=None,
             input_shape=None,
             pooling=None,
             classes=7):

    img_input = Input(shape=input_shape)

    
    bn_axis = 3
    x = Lambda(lambda img : tf.image.resize(img, (224, 224)))(img_input)
    x = ZeroPadding2D(padding=(3, 3), name='conv1_pad')(x)
    
    x = Conv2D(64, (7, 7),
                      strides=(2, 2),
                      padding='valid',
                      use_bias=False,
                      kernel_initializer='he_normal',
                      name='conv0')(x)
    x = BatchNormalization(axis=bn_axis, name='bn0')(x)
    x = Activation('relu', name='relu0')(x)
    x = ZeroPadding2D(padding=(1, 1), name='pool1_pad')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), name='pooling0')(x)

    x = conv_block(x, 64, stage=0, block=0, strides=(1, 1))
    x = identity_block(x, 64, stage=0, block=1)
    x = identity_block(x, 64, stage=0, block=2)
    M = masking(x, 64, stage=0, block=92, depth=4)
    M = Lambda(lambda m : 1 + m)(M)
    x = Multiply()([x, M])

    x = conv_block(x, 128, stage=1, block=0)
    x = identity_block(x, 128, stage=1, block=1)
    x = identity_block(x, 128, stage=1, block=2)
    x = identity_block(x, 128, stage=1, block=3)
    M = masking(x, 128, stage=1, block=93, depth=3)
    M = Lambda(lambda m : 1 + m)(M)
    x = Multiply()([x, M])

    x = conv_block(x, 256, stage=2, block=0)
    x = identity_block(x, 256, stage=2, block=1)
    x = identity_block(x, 256, stage=2, block=2)
    x = identity_block(x, 256, stage=2, block=3)
    x = identity_block(x, 256, stage=2, block=4)
    x = identity_block(x, 256, stage=2, block=5)
    M = masking(x, 256, stage=2, block=95, depth=2)
    M = Lambda(lambda m : 1 + m)(M)
    x = Multiply()([x, M])

    x = conv_block(x, 512, stage=3, block=0)
    x = identity_block(x, 512, stage=3, block=1)
    x = identity_block(x, 512, stage=3, block=2)
    M = masking(x, 512, stage=3, block=92, depth=1)
    M = Lambda(lambda m : 1 + m)(M)
    x = Multiply()([x, M])

    x = BatchNormalization(axis=3, name='bn1')(x)
    x = Activation('relu', name='relu1')(x)
    # x = AveragePooling2D(pool_size=(2,2), padding='same', name='avg_pool')(x)
    # x = Flatten()(x)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.4)(x)
    x = Dense(classes, activation='softmax', name='fc7')(x)

    # Create model.
    model = Model(img_input, x, name='resnet')

    weights_path = get_file(
        'resnet34_imagenet_1000_no_top.h5',
        'https://github.com/qubvel/classification_models/releases/download/0.0.1/resnet34_imagenet_1000_no_top.h5',
        cache_subdir='models',
        md5_hash='8caaa0ad39d927cb8ba5385bf945d582'
    )
    model.load_weights(weights_path, by_name=True)

    return model

model = ResMaskingNet(input_shape=(48, 48, 3), classes=7)
model.summary()


In [5]:
from    keras.utils.vis_utils    import plot_model
plot_model(model, to_file="model.png",show_shapes=True)

In [9]:
model = ResMaskingNet(input_shape=(48, 48, 3), classes=7)

for layer in model.layers[:-1]:
  layer.trainable = False

lr_reducer = ReduceLROnPlateau(monitor='val_accuracy', factor=0.9, patience=3, verbose=1)
checkpointer = ModelCheckpoint("./residual_masking.h5", monitor='val_accuracy', verbose=1, save_best_only=True)

model.compile(optimizer=SGD(lr=1e-2, decay=1e-6, momentum=0.9, nesterov=True), loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(aug.flow(training_pixels, training_labels, 48), steps_per_epoch=len(training_pixels) // 48, epochs=15, 
                             validation_data= (validation_pixels, validation_labels),  callbacks=[lr_reducer, checkpointer])



for layer in model.layers[:-1]:
  layer.trainable = True

model.compile(optimizer=SGD(lr=1e-2, decay=1e-6, momentum=0.9, nesterov=True), loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(aug.flow(training_pixels, training_labels, 48), steps_per_epoch=len(training_pixels) // 48, epochs=70, 
                             validation_data= (validation_pixels, validation_labels),  callbacks=[lr_reducer, checkpointer])

In [15]:
train_score = model.evaluate(training_pixels, training_labels, verbose=0)
print('Train loss:', train_score[0])
print('Train accuracy:', 100*train_score[1])
 
test_score = model.evaluate(validation_pixels, validation_labels, verbose=0)
print('Test loss:', test_score[0])
print('Test accuracy:', 100*test_score[1])


In [16]:
model.save('RNNMASK.hdf5') 

In [10]:
y_pred = model.predict(validation_pixels)
for i in range(len(y_pred)):
        max_value=max(y_pred[i])
        for j in range(len(y_pred[i])):
            if max_value==y_pred[i][j]:
                y_pred[i][j]=1
            else:
                y_pred[i][j]=0

In [11]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
con_mat = confusion_matrix(validation_labels.argmax(axis=1), y_pred.argmax(axis=1))

con_mat_norm = con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis]     # 归一化
con_mat_norm = np.around(con_mat_norm, decimals=4)
figure = plt.figure(figsize=(18, 18))
sns.heatmap(con_mat_norm, annot=True, cmap='summer')
EMOTIONS = ["angry", "disgust", "scared", "happy", "sad", "surprised", "neutral"]
emotion ={0:'Angry',1:'Disgust',2:'Fear',3:'Happy',4:'Sad',5:'Surprise',6:'Neutral'}
xlocations = np.array(range(len(EMOTIONS)))
plt.xticks(range(7), ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral'], rotation=45)
plt.yticks(range(7), ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral'], rotation=45)
plt.ylim(0, 10)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.savefig(fname="pic.png",figsize=[10,10])
plt.show()